In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime
from pypfopt import EfficientFrontier, risk_models, expected_returns
from pypfopt.discrete_allocation import DiscreteAllocation

# 1. Stock Selection
# Selected SGX stocks (example - you should replace with your own selection)
stocks = ['C52.SI', 'D05.SI', 'O39.SI', 'U11.SI', 'Z74.SI', 'S68.SI', 'C38U.SI']  # Example tickers

def download_stock_data(tickers, start_date, end_date):
    df = pd.DataFrame()
    for ticker in tickers:
        stock_data = yf.download(ticker, start=start_date, end=end_date)['Adj Close']
        df[ticker] = stock_data
    return df

# Download data from 2015 to 2023
train_start = '2015-01-01'
train_end = '2022-12-31'
test_start = '2023-01-01'
test_end = '2023-12-31'

# Get training and testing data
train_prices = download_stock_data(stocks, train_start, train_end)
test_prices = download_stock_data(stocks, test_start, test_end)

# Check for missing values
print("Missing values in training data:")
print(train_prices.isnull().sum())\

yes


In [ ]:
# Remove stocks with too many missing values if necessary
train_prices = train_prices.dropna(axis=1, thresh=len(train_prices)*0.9)
stocks = list(train_prices.columns)

# 4. Calculate expected returns and covariance matrix
mu = expected_returns.mean_historical_return(train_prices)
S = risk_models.CovarianceShrinkage(train_prices).ledoit_wolf()

In [ ]:
# 5. Optimize portfolio for maximum Sharpe ratio
ef = EfficientFrontier(mu, S)
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
print("\nOptimal weights:")
print(cleaned_weights)

In [ ]:


# 6a. Backtest using weight multiplication
# Calculate daily returns
train_returns = train_prices.pct_change().dropna()
test_returns = test_prices.pct_change().dropna()

# Calculate portfolio returns
portfolio_returns = pd.Series(0, index=test_returns.index)
for stock in stocks:
    portfolio_returns += test_returns[stock] * cleaned_weights[stock]

# Calculate performance metrics
total_return = (1 + portfolio_returns).prod() - 1
annual_volatility = portfolio_returns.std() * np.sqrt(252)
sharpe_ratio = total_return / annual_volatility  # Simplified Sharpe ratio

print("\nBacktest Results (Weight Multiplication):")
print(f"Total Return: {total_return:.4f}")
print(f"Annual Volatility: {annual_volatility:.4f}")
print(f"Sharpe Ratio: {sharpe_ratio:.4f}")

# 6b. Backtest using discrete allocation
portfolio_value = 1000000  # $1M portfolio
latest_prices = train_prices.iloc[-1]
da = DiscreteAllocation(cleaned_weights, latest_prices, portfolio_value)
allocation, leftover = da.greedy_portfolio()

print("\nDiscrete Allocation:")
print(allocation)
print(f"Funds remaining: ${leftover:.2f}")

# Calculate portfolio value over time
portfolio_value_series = pd.Series(0, index=test_prices.index)
for stock, shares in allocation.items():
    portfolio_value_series += test_prices[stock] * shares

# Calculate discrete allocation returns
discrete_returns = portfolio_value_series.pct_change().dropna()
total_return_discrete = (1 + discrete_returns).prod() - 1
annual_vol_discrete = discrete_returns.std() * np.sqrt(252)
sharpe_ratio_discrete = total_return_discrete / annual_vol_discrete

print("\nBacktest Results (Discrete Allocation):")
print(f"Total Return: {total_return_discrete:.4f}")
print(f"Annual Volatility: {annual_vol_discrete:.4f}")
print(f"Sharpe Ratio: {sharpe_ratio_discrete:.4f}")

In [ ]:
# 7. Portfolio optimization with sector constraints
# Define sector mappings (example)
sector_mapper = {
    'C52.SI': 'Finance',
    'D05.SI': 'Finance',
    'O39.SI': 'Industrial',
    'U11.SI': 'Technology',
    'Z74.SI': 'Consumer',
    'S68.SI': 'Technology',
    'C38U.SI': 'REIT'
}

# First set of sector constraints
sector_lower = {
    'Finance': 0.2,
    'Industrial': 0.1,
    'Technology': 0.15,
    'Consumer': 0.1,
    'REIT': 0.1
}

sector_upper = {
    'Finance': 0.4,
    'Industrial': 0.3,
    'Technology': 0.3,
    'Consumer': 0.25,
    'REIT': 0.25
}

# New optimization with sector constraints
ef_sector = EfficientFrontier(mu, S)
ef_sector.add_sector_constraints(sector_mapper, sector_lower, sector_upper)
weights_sector = ef_sector.max_sharpe()
cleaned_weights_sector = ef_sector.clean_weights()

print("\nOptimal weights with sector constraints:")
print(cleaned_weights_sector)

# Backtest the sector-constrained portfolio
portfolio_returns_sector = pd.Series(0, index=test_returns.index)
for stock in stocks:
    portfolio_returns_sector += test_returns[stock] * cleaned_weights_sector[stock]

# Calculate performance metrics
total_return_sector = (1 + portfolio_returns_sector).prod() - 1
annual_vol_sector = portfolio_returns_sector.std() * np.sqrt(252)
sharpe_ratio_sector = total_return_sector / annual_vol_sector

print("\nBacktest Results (Sector Constrained):")
print(f"Total Return: {total_return_sector:.4f}")
print(f"Annual Volatility: {annual_vol_sector:.4f}")
print(f"Sharpe Ratio: {sharpe_ratio_sector:.4f}")

# 8. Compare with benchmark (STI ETF)
benchmark_ticker = 'ES3.SI'  # STI ETF
benchmark_prices = yf.download(benchmark_ticker, start=test_start, end=test_end)['Adj Close']
benchmark_returns = benchmark_prices.pct_change().dropna()

benchmark_total_return = (1 + benchmark_returns).prod() - 1
benchmark_vol = benchmark_returns.std() * np.sqrt(252)
benchmark_sharpe = benchmark_total_return / benchmark_vol

print("\nBenchmark Performance:")
print(f"Total Return: {benchmark_total_return:.4f}")
print(f"Annual Volatility: {benchmark_vol:.4f}")
print(f"Sharpe Ratio: {benchmark_sharpe:.4f}")

# Plotting for visualization
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot((1 + portfolio_returns).cumprod(), label='Base Portfolio')
plt.plot((1 + portfolio_returns_sector).cumprod(), label='Sector Constrained')
plt.plot((1 + benchmark_returns).cumprod(), label='STI ETF')
plt.title('Portfolio Performance Comparison')
plt.xlabel('Date')
plt.ylabel('Cumulative Return')
plt.legend()
plt.grid(True)
plt.show()